In [22]:
## Class Definitions

class Quote:
    def __init__(self, quote, category):
        self.quote = quote
        self.category = category
        

In [38]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/reduced_quotes.csv')

df.dropna()

quotes = []

df['processed_category'] = df['category'].apply(lambda x: ' '.join(str(x).split(', ')))

for _, row in df.iterrows():
    quotes.append(Quote(row['quote'], row['processed_category']) )


In [46]:

from sklearn.model_selection import train_test_split

print(len(quotes))

training, test = train_test_split(quotes, test_size=0.33, random_state=42, shuffle=True)

print(len(training))

x_train = [q.quote for q in training]
y_train = [q.category for q in training]

x_train = ["" if pd.isna(text) else str(text) for text in x_train]


x_test = [q.quote for q in test]
y_test = [q.category for q in test]

print(x_test[0])
print(y_test[0])


499709
334805
The sting of her abandonment had not lessened through the years, and I suspected it would never go away. Occasionally, I could see agony in her eyes, the shadows that flickered in the background. If I could, I'd take her pain and make it my own. I'd swallow it like a bitter pill and live with the consequences.
love relationship suffering


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
train_x_vectors = tfidf_vectorizer.fit_transform(x_train)

In [52]:
print(train_x_vectors.shape)
print(train_x_vectors[0].toarray())
print(tfidf_vectorizer.get_feature_names_out())


(334805, 127983)
[[0. 0. 0. ... 0. 0. 0.]]
['00' '000' '0000' ... 'ﬂowersraised' 'ﬂows' 'ﬂush']
